# PCT Training

Trains Menghao PCT model from the Point-Transformers implementation: https://github.com/qq456cvb/Point-Transformers

Allows using original data, as well as PAPNet partial data.

## Env prep

First, mount Google drive, clone branch, change working directory, and install dependencies. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!git clone --recursive --single-branch --branch pct-dataloader https://github.com/DavidClaszen/pointcloud-bench /content/pointcloud-bench
%cd /content/pointcloud-bench
%pip install -r envs/pct/requirements.txt

In [ ]:
# Check for CUDA/GPU
import torch, sys
print(sys.version)
print('Torch:', torch.__version__, 'CUDA:', torch.version.cuda, 'GPU:', torch.cuda.is_available())

## Datasets

Because Google Drive is terribly slow when dealing with lots of small files, unzip an archived version into /content/, then add symlinks. Make sure the files aren't nested inside another folder inside the .zip file.

- PAPNet partial data: requires a `partialmodelnet40.zip` file
- Point-Transformers normalized ModelNet40: requires a `modelnet40_normal_resampled.zip` file

In [ ]:
# PAPNet partials
ZIP = '/content/drive/MyDrive/pointcloud-bench/datasets/partialmodelnet40.zip'
DEST = '/content/modelnet40'
!mkdir -p "{DEST}"
!unzip -q -n "{ZIP}" -d "{DEST}"
!find "{DEST}" -type f | wc -l

In [ ]:
# Original data used by Point-Transformers, from ShapeNet
ZIP = '/content/drive/MyDrive/pointcloud-bench/datasets/modelnet40_normal_resampled.zip'
DEST = '/content/modelnet40'
!unzip -q -n '{ZIP}' -d '{DEST}'
!find '{DEST}' -type f | wc -l

# Model Training




In [ ]:
%cd /content/pointcloud-bench/repos/Point-Transformers
!python train_cls.py --help

Model arguments can be appended to the command, such as the following to run a single epoch for Hengshuang.

`!python train_cls.py epoch=1 model=Hengshuang`

The Point-Transformers repo makes available three implementations. On an L4, one epoch took roughly:

- Menghao: 2-3 minutes
- Hengshuang: 5-6 minutes
- Nico: 4-5 minutes

Implementing Automatic Mixed Precision cut down these training times significantly, almost half for Menghao. 

For default data, set `use_papnet_loader` to `False` or just use the defaults. Must be `True` for PAPNet style partial data.

In [ ]:
# Train Menghao
!python train_cls.py model=Menghao use_papnet_loader=True batch_size=128 learning_rate=0.001 epoch=250 workers=2 data_path=../../../modelnet40/

In [ ]:
# Zip Point-Transformers logs, included last best model, to continue training later
!zip -r results.zip ./repos/Point-Transformers/log/cls/Menghao/